# Agents

AutoGen AgentChat provides a set of preset Agents, each with variations in how an agent might respond to messages.
All agents share the following attributes and methods:

- `~autogen_agentchat.agents.BaseChatAgent.name`: The unique name of the agent.
- `~autogen_agentchat.agents.BaseChatAgent.description`: The description of the agent in text.
- `~autogen_agentchat.agents.BaseChatAgent.on_messages`: Send the agent a sequence of messages that subclass `~autogen_agentchat.messages.BaseChatMessage` and get a `~autogen_agentchat.base.Response`. **It is important to note that agents are expected to be stateful and this method is expected to be called with new messages, not the complete history**.
- `~autogen_agentchat.agents.BaseChatAgent.on_messages_stream`: Same as `~autogen_agentchat.agents.BaseChatAgent.on_messages` but returns an iterator of messages that subclass `~autogen_agentchat.messages.BaseAgentEvent` or `~autogen_agentchat.messages.BaseChatMessage` followed by a `~autogen_agentchat.base.Response` as the last item.
- `~autogen_agentchat.agents.BaseChatAgent.on_reset`: Reset the agent to its initial state.
- `~autogen_agentchat.agents.BaseChatAgent.run` and `~autogen_agentchat.agents.BaseChatAgent.run_stream`: convenience methods that call `~autogen_agentchat.agents.BaseChatAgent.on_messages` and `~autogen_agentchat.agents.BaseChatAgent.on_messages_stream` respectively but offer the same interface as [Teams](./teams.ipynb).

See `autogen_agentchat.messages` for more information on AgentChat message types.


## Assistant Agent

`~autogen_agentchat.agents.AssistantAgent` is a built-in agent that
uses a language model and has the ability to use tools.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import StructuredMessage, TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient



In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv(override=True)  # Load environment variables from .env file

# Load environment variables
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=deployment_name,
    model=model_name,
    api_version=api_version,
    azure_endpoint=base_url,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=api_key, # For key-based authentication.
)

In [ ]:
# Define a tool that searches the web for information.
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


agent = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
)


## Getting Responses

We can use the `~autogen_agentchat.agents.AssistantAgent.on_messages` method to get the agent response to a given message.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Find information on AutoGen", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

The call to the `~autogen_agentchat.agents.AssistantAgent.on_messages` method
returns a `~autogen_agentchat.base.Response`
that contains the agent's final response in the `~autogen_agentchat.base.Response.chat_message` attribute,
as well as a list of inner messages in the `~autogen_agentchat.base.Response.inner_messages` attribute,
which stores the agent's "thought process" that led to the final response.

```{note}
It is important to note that `~autogen_agentchat.agents.AssistantAgent.on_messages`
will update the internal state of the agent -- it will add the messages to the agent's
history. So you should call this method with new messages.
**You should not repeatedly call this method with the same messages or the complete history.**
```

```{note}
Unlike in v0.2 AgentChat, the tools are executed by the same agent directly within
the same call to `~autogen_agentchat.agents.AssistantAgent.on_messages`.
By default, the agent will return the result of the tool call as the final response.
```

You can also call the `~autogen_agentchat.agents.BaseChatAgent.run` method, which is a convenience method that calls `~autogen_agentchat.agents.BaseChatAgent.on_messages`. 
It follows the same interface as [Teams](./teams.ipynb) and returns a `~autogen_agentchat.base.TaskResult` object.

## Multi-Modal Input

The `~autogen_agentchat.agents.AssistantAgent` can handle multi-modal input
by providing the input as a `~autogen_agentchat.messages.MultiModalMessage`.

In [ ]:
from io import BytesIO

import PIL
import requests
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import Image

# Create a multi-modal message with random image and text.
pil_image = PIL.Image.open(BytesIO(requests.get("https://picsum.photos/300/200").content))
img = Image(pil_image)
multi_modal_message = MultiModalMessage(content=["Can you describe the content of this image?", img], source="user")
img

In [ ]:
# Use asyncio.run(...) when running in a script.
response = await agent.on_messages([multi_modal_message], CancellationToken())
print(response.chat_message)

You can also use `~autogen_agentchat.messages.MultiModalMessage` as a `task`
input to the `~autogen_agentchat.agents.BaseChatAgent.run` method.

## Streaming Messages

We can also stream each message as it is generated by the agent by using the
`~autogen_agentchat.agents.AssistantAgent.on_messages_stream` method,
and use `~autogen_agentchat.ui.Console` to print the messages
as they appear to the console.

In [ ]:
async def assistant_run_stream() -> None:
    # Option 1: read each message from the stream (as shown in the previous example).
    # async for message in agent.on_messages_stream(
    #     [TextMessage(content="Find information on AutoGen", source="user")],
    #     cancellation_token=CancellationToken(),
    # ):
    #     print(message)

    # Option 2: use Console to print all messages as they appear.
    await Console(
        agent.on_messages_stream(
            [TextMessage(content="Find information on AutoGen", source="user")],
            cancellation_token=CancellationToken(),
        ),
        output_stats=True,  # Enable stats printing.
    )


# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()

The `~autogen_agentchat.agents.AssistantAgent.on_messages_stream` method
returns an asynchronous generator that yields each inner message generated by the agent,
with the final item being the response message in the `~autogen_agentchat.base.Response.chat_message` attribute.

From the messages, you can observe that the assistant agent utilized the `web_search` tool to
gather information and responded based on the search results.

You can also use `~autogen_agentchat.agents.BaseChatAgent.run_stream` to get the same streaming behavior as `~autogen_agentchat.agents.BaseChatAgent.on_messages_stream`. It follows the same interface as [Teams](./teams.ipynb).

## Using Tools

Large Language Models (LLMs) are typically limited to generating text or code responses. 
However, many complex tasks benefit from the ability to use external tools that perform specific actions,
such as fetching data from APIs or databases.

To address this limitation, modern LLMs can now accept a list of available tool schemas 
(descriptions of tools and their arguments) and generate a tool call message. 
This capability is known as **Tool Calling** or **Function Calling** and 
is becoming a popular pattern in building intelligent agent-based applications.
Refer to the documentation from [OpenAI](https://platform.openai.com/docs/guides/function-calling) 
and [Anthropic](https://docs.anthropic.com/en/docs/build-with-claude/tool-use) for more information about tool calling in LLMs.

In AgentChat, the `~autogen_agentchat.agents.AssistantAgent` can use tools to perform specific actions.
The `web_search` tool is one such tool that allows the assistant agent to search the web for information.
A custom tool can be a Python function or a subclass of the `~autogen_core.tools.BaseTool`.

```{note}
For how to use model clients directly with tools, refer to the [Tools](../../core-user-guide/components/tools.ipynb) section
in the Core User Guide.
```

By default, when `~autogen_agentchat.agents.AssistantAgent` executes a tool,
it will return the tool's output as a string in `~autogen_agentchat.messages.ToolCallSummaryMessage` in its response.
If your tool does not return a well-formed string in natural language, you
can add a reflection step to have the model summarize the tool's output,
by setting the `reflect_on_tool_use=True` parameter in the `~autogen_agentchat.agents.AssistantAgent` constructor.

### Built-in Tools

AutoGen Extension provides a set of built-in tools that can be used with the Assistant Agent.
Head over to the [API documentation](../../../reference/index.md) for all the available tools
under the `autogen_ext.tools` namespace. For example, you can find the following tools:

- `~autogen_ext.tools.graphrag`: Tools for using GraphRAG index.
- `~autogen_ext.tools.http`: Tools for making HTTP requests.
- `~autogen_ext.tools.langchain`: Adaptor for using LangChain tools.
- `~autogen_ext.tools.mcp`: Tools for using Model Chat Protocol (MCP) servers.

### Function Tool

The `~autogen_agentchat.agents.AssistantAgent` automatically
converts a Python function into a `~autogen_core.tools.FunctionTool`
which can be used as a tool by the agent and automatically generates the tool schema
from the function signature and docstring.

The `web_search_func` tool is an example of a function tool.
The schema is automatically generated.

In [ ]:
from autogen_core.tools import FunctionTool


# Define a tool using a Python function.
async def web_search_func(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


# This step is automatically performed inside the AssistantAgent if the tool is a Python function.
web_search_function_tool = FunctionTool(web_search_func, description="Find information on the web")
# The schema is provided to the model during AssistantAgent's on_messages call.
web_search_function_tool.schema

### Model Context Protocol Tools

The `~autogen_agentchat.agents.AssistantAgent` can also use tools that are
served from a Model Context Protocol (MCP) server
using `~autogen_ext.tools.mcp.mcp_server_tools`.

In [ ]:
# pip install mcp-server-fetch

In [ ]:
import asyncio

from pathlib import Path
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=deployment_name,
    model=model_name,
    api_version=api_version,
    azure_endpoint=base_url,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=api_key, # For key-based authentication.
)

# Setup server params for local filesystem access
desktop = str(Path.cwd())
server_params = StdioServerParams(
    command="npx", args=["-y", "@modelcontextprotocol/server-filesystem", desktop]
)

# Get all available tools from the server
tools = await mcp_server_tools(server_params)

# Create an agent that can use all the tools
agent = AssistantAgent(
    name="file_manager",
    model_client=az_model_client,
    tools=tools,  # type: ignore
)

# The agent can now use any of the filesystem tools
await Console(agent.run_stream(task="Create a file called test.txt with some content", cancellation_token=CancellationToken()))

### Parallel Tool Calls

Some models support parallel tool calls, which can be useful for tasks that require multiple tools to be called simultaneously.
By default, if the model client produces multiple tool calls, `~autogen_agentchat.agents.AssistantAgent`
will call the tools in parallel.

You may want to disable parallel tool calls when the tools have side effects that may interfere with each other, or,
when agent behavior needs to be consistent across different models.
This should be done at the model client level.

For `~autogen_ext.models.openai.OpenAIChatCompletionClient` and `~autogen_ext.models.openai.AzureOpenAIChatCompletionClient`,
set `parallel_tool_calls=False` to disable parallel tool calls.

In [ ]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=deployment_name,
    model=model_name,
    api_version=api_version,
    azure_endpoint=base_url,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=api_key, # For key-based authentication.
    parallel_tool_calls=False,  # type: ignore
)

agent_no_parallel_tool_call = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
)

## Running an Agent in a Loop

The `~autogen_agentchat.agents.AssistantAgent` executes one
step at a time: one model call, followed by one tool call (or parallel tool calls), and then
an optional reflection.

To run it in a loop, for example, running it until it stops producing
tool calls, please refer to [Single-Agent Team](./teams.ipynb#single-agent-team).

## Structured Output

Structured output allows models to return structured JSON text with pre-defined schema
provided by the application. Different from JSON-mode, the schema can be provided
as a [Pydantic BaseModel](https://docs.pydantic.dev/latest/concepts/models/)
class, which can also be used to validate the output.

Once you specify the base model class in the `output_content_type` parameter
of the `~autogen_agentchat.agents.AssistantAgent` constructor,
the agent will respond with a `~autogen_agentchat.messages.StructuredMessage`
whose `content`'s type is the type of the base model class.

This way, you can integrate agent's response directly into your application
and use the model's output as a structured object.

```{note}
When the `output_content_type` is set, it by default requires the agent to reflect on the tool use
and return the a structured output message based on the tool call result.
You can disable this behavior by setting `reflect_on_tool_use=False` explictly.
```

Structured output is also useful for incorporating Chain-of-Thought
reasoning in the agent's responses.
See the example below for how to use structured output with the assistant agent.

In [ ]:
from typing import Literal

from pydantic import BaseModel


# The response format for the agent as a Pydantic base model.
class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["happy", "sad", "neutral"]

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=deployment_name,
    model=model_name,
    api_version=api_version,
    azure_endpoint=base_url,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=api_key, # For key-based authentication.
)

agent = AssistantAgent(
    "assistant",
    model_client=az_model_client,
    system_message="Categorize the input as happy, sad, or neutral following the JSON format.",
    # Define the output content type of the agent.
    output_content_type=AgentResponse,
)

result = await Console(agent.run_stream(task="I am happy."))

# Check the last message in the result, validate its type, and print the thoughts and response.
assert isinstance(result.messages[-1], StructuredMessage)
assert isinstance(result.messages[-1].content, AgentResponse)
print("Thought: ", result.messages[-1].content.thoughts)
print("Response: ", result.messages[-1].content.response)
await az_model_client.close()

## Streaming Tokens

You can stream the tokens generated by the model client by setting `model_client_stream=True`.
This will cause the agent to yield `~autogen_agentchat.messages.ModelClientStreamingChunkEvent` messages
in `~autogen_agentchat.agents.BaseChatAgent.on_messages_stream` and `~autogen_agentchat.agents.BaseChatAgent.run_stream`.

The underlying model API must support streaming tokens for this to work.
Please check with your model provider to see if this is supported.

In [ ]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=deployment_name,
    model=model_name,
    api_version=api_version,
    azure_endpoint=base_url,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=api_key, # For key-based authentication.
)

streaming_assistant = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    system_message="You are a helpful assistant.",
    model_client_stream=True,  # Enable streaming tokens.
)

# Use an async function and asyncio.run() in a script.
async for message in streaming_assistant.on_messages_stream(  # type: ignore
    [TextMessage(content="Name two cities in South America", source="user")],
    cancellation_token=CancellationToken(),
):
    print(message)

You can see the streaming chunks in the output above.
The chunks are generated by the model client and are yielded by the agent as they are received.
The final response, the concatenation of all the chunks, is yielded right after the last chunk.

Similarly, `~autogen_agentchat.agents.BaseChatAgent.run_stream` will also yield the same streaming chunks,
followed by a full text message right after the last chunk.

In [ ]:
async for message in streaming_assistant.run_stream(task="Name two cities in North America."):  # type: ignore
    print(message)

## Using Model Context

`~autogen_agentchat.agents.AssistantAgent` has a `model_context`
parameter that can be used to pass in a `~autogen_core.model_context.ChatCompletionContext`
object. This allows the agent to use different model contexts, such as
`~autogen_core.model_context.BufferedChatCompletionContext` to
limit the context sent to the model.

By default, `~autogen_agentchat.agents.AssistantAgent` uses
the `~autogen_core.model_context.UnboundedChatCompletionContext`
which sends the full conversation history to the model. To limit the context
to the last `n` messages, you can use the `~autogen_core.model_context.BufferedChatCompletionContext`.
To limit the context by token count, you can use the
`~autogen_core.model_context.TokenLimitedChatCompletionContext`.

In [ ]:
from autogen_core.model_context import BufferedChatCompletionContext

# Create an agent that uses only the last 5 messages in the context to generate responses.
agent = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
    model_context=BufferedChatCompletionContext(buffer_size=5),  # Only use the last 5 messages in the context.
)

## Other Preset Agents

The following preset agents are available:

- `~autogen_agentchat.agents.UserProxyAgent`: An agent that takes user input returns it as responses.
- `~autogen_agentchat.agents.CodeExecutorAgent`: An agent that can execute code.
- `~autogen_ext.agents.openai.OpenAIAssistantAgent`: An agent that is backed by an OpenAI Assistant, with ability to use custom tools.
- `~autogen_ext.agents.web_surfer.MultimodalWebSurfer`: A multi-modal agent that can search the web and visit web pages for information.
- `~autogen_ext.agents.file_surfer.FileSurfer`: An agent that can search and browse local files for information.
- `~autogen_ext.agents.video_surfer.VideoSurfer`: An agent that can watch videos for information.

## Next Step

Having explored the usage of the `~autogen_agentchat.agents.AssistantAgent`, we can now proceed to the next section to learn about the teams feature in AgentChat.


<!-- ## CodingAssistantAgent

Generates responses (text and code) using an LLM upon receipt of a message. It takes a `system_message` argument that defines or sets the tone for how the agent's LLM should respond. 

```python

writing_assistant_agent = CodingAssistantAgent(
    name="writing_assistant_agent",
    system_message="You are a helpful assistant that solve tasks by generating text responses and code.",
    model_client=model_client,
)
`

We can explore or test the behavior of the agent by sending a message to it using the  `~autogen_agentchat.agents.BaseChatAgent.on_messages`  method. 

```python
result = await writing_assistant_agent.on_messages(
    messages=[
        TextMessage(content="What is the weather right now in France?", source="user"),
    ],
    cancellation_token=CancellationToken(),
)
print(result) -->